# **Build RAG from Scratch**

- Step 1: Document Loading and Preprocessing
- Step 2: Text Splitting and Chunking
- Step 3: Embedding Generation (Indexing)
- Step 4: Vector Database Storage
- Step 5: Query Processing and Retrieval: Perform similarity search (Dense Retrieval) and retrieve top-K candidates
- Step 6: Reranking: Initialize cross-encoder model, rerank retrieved candidates and filter to top-N results
- Step 7: Context Preparation for LLM: Format retrieved chunks
- Step 8: LLM Generation (Optional): Initialize LLM and generate response with context
- _Evaluation and Testing_: Test queries, Evaluation metrics and Performance analysis

# Raw Document Analysis (Optional)

In [1]:
def word_count(file_name: str) -> None:
    # More accurate word count for Chinese
    with open(file_name, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Count Chinese characters (each Chinese char ≈ 1 word)
    chinese_chars = len([c for c in content if '\u4e00' <= c <= '\u9fff'])
    # Count English words
    english_words = len([w for w in content.split() if any('a' <= c.lower() <= 'z' for c in w)])
    
    print(f"Total characters: {len(content)}")
    print(f"Chinese characters: {chinese_chars} (≈ words)")
    print(f"English words: {english_words}")
    print(f"Approximate total words: {chinese_chars + english_words}")

word_count("cn1.md")

Total characters: 8085
Chinese characters: 6433 (≈ words)
English words: 24
Approximate total words: 6457


# Part 1: Splitting and Chunking Strategy

In [2]:
from typing import List
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_into_chunks(
    doc_file: str, 
    chunk_size: int = 800, 
    chunk_overlap: int = 150
) -> List[str]:
    with open(doc_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n## ", "\n# ", "\n\n", "\n", "。", "!", "?", ";", " ", ""]
    )
    return text_splitter.split_text(content)

chunks = split_into_chunks("cn1.md")
print(f"Created {len(chunks)} chunks\n")


Created 15 chunks



# Part 2: Indexing and Storage
**ChromaDB handles _cosine_ similarity behind the scenes** - that's the whole point of using a vector database instead of computing similarities manually. `client.create_collection(..., metadata={"hnsw:space": "cosine"})`

Embedding Model Choice:
- `shibing624/text2vec-base-chinese` (good for Chinese and used in `rag0`)
- `BAAI/bge-base-zh-v1.5` (better Chinese performance)
- `moka-ai/m3e-base` (multilingual Chinese-English)

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
from typing import List

# ======================================== 
# Initialize models
# ========================================
embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")


# ======================================== 
# Create ChromaDB with correct settings
# ========================================
def create_db():
    client = chromadb.PersistentClient(path="./chroma_db")

    # If you want to GUARANTEE cosine space, delete+recreate:
    try:
        client.delete_collection(name="default")
        print("Deleted old collection")
    except Exception as e:
        # ok if it doesn't exist; still good to know unexpected errors
        print(f"(delete_collection) {e}")

    collection = client.create_collection(
        name="default",
        metadata={"hnsw:space": "cosine"}
    )

    print(f"Created collection with metadata: {collection.metadata}")
    return collection

chromadb_collection = create_db()


# ======================================== 
# Embed
# ========================================
def embed_chunk(chunk: str) -> List[float]:
    emb = embedding_model.encode(chunk)  # numpy array
    return emb.tolist()


# ======================================== 
# Store with metadata
# ========================================
def save_embeddings(
    collection,
    chunks: List[str],
    embeddings: List[List[float]],
    source_file: str,
) -> None:
    if not chunks:
        raise ValueError("chunks is empty")

    if len(chunks) != len(embeddings):
        raise ValueError(f"chunks ({len(chunks)}) and embeddings ({len(embeddings)}) length mismatch")
    
    if not source_file:
        return ValueError("file is empty")

    # Use stable IDs that won't collide across different files
    ids = [f"{source_file}:{i}" for i in range(len(chunks))]

    metadatas = [
        {
            "chunk_id": i,
            "source": source_file,
            "chunk_length": len(chunk),
            "chunk_index": i,
        }
        for i, chunk in enumerate(chunks)
    ]

    # Use upsert so reruns don't explode on duplicate ids
    collection.upsert(
        documents=chunks,
        embeddings=embeddings,
        ids=ids,
        metadatas=metadatas,
    )

    print(f"Saved {len(chunks)} chunks to ChromaDB")

# embeddings
embeddings = [embed_chunk(c) for c in chunks]
print(f"Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {len(embeddings[0])}")

save_embeddings(chromadb_collection, chunks, embeddings, "cn1.md")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: shibing624/text2vec-base-chinese
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Deleted old collection
Created collection with metadata: {'hnsw:space': 'cosine'}
Generated 15 embeddings
Embedding dimension: 768
Saved 15 chunks to ChromaDB


# Part 3: Retrieval

Step 1 (in part 3): **Bi-encoder** Retrieval (Fast but less accurate)
- Encode query and docs separately → Compare vectors with cosine similarity
- Get top-K candidates i.e., 5, 10 or 20
- Time: ~0.03s

Step 2 (in part 4): **Cross-encoder** Reranking (Slow but very accurate)  
- Process (query + doc) together with attention → Get relevance score
- Rerank top-K → top 3 results
- Time: ~2.5s

>**Q**: Why not use cross-encoder only?
>
>**A**: Too expensive! Cross-encoder must process ALL documents (10K docs = 
   500s per query). Bi-encoder caches document embeddings, so only needs 
   to encode query (0.03s). Result: 167× speedup.

Key Insight: Bi-encoder casts wide net efficiently, cross-encoder filters to highest quality. But if retrieval can return high quality results, the rerank is not a must.

In [4]:
def retrieve(query: str, top_k: int = 5, score_threshold = None):
    query_embedding = embed_chunk(query)
    
    results = chromadb_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        include=['documents', 'distances', 'metadatas']
    )
    
    if not results['documents'][0]:
        print("No results found!")
        return []
    
    retrieved = []
    for i, (doc, dist, meta) in enumerate(zip(
        results['documents'][0],
        results['distances'][0],
        results['metadatas'][0]
    )):
        if score_threshold is None or dist <= score_threshold:
            retrieved.append({
                'document': doc,
                'distance': dist,  # This is actually cosine distance (1 - cosine_similarity)
                'similarity': 1 / (1 + dist), # Manual conversion to similarity score from 0 to 1 - more intuitive to read
                'metadata': meta,
                'rank': i
            })
    
    print(f"Retrieved {len(retrieved)}/{top_k} chunks")
    return retrieved

## Retrieval Testing

In [5]:
# query = "哈利波特用了什么魔法打败了索伦？"
query = "哈利用什么魔法打败了索伦？"
results = retrieve(query, top_k=5, score_threshold=None)

def print_top_k_result(results):
    print("\n" + "="*70)
    print("Results for ", query)
    print("="*70)
    for i, r in enumerate(results):
        print(f"\nRank {i+1}:")
        print(f"    Distance: {r['distance']:.3f}")  # Should now be 0.0-2.0 range
        print(f"    Similarity: {r['similarity']:.3f}")
        print(f"    Text: {r['document'][:80]}...")

print_top_k_result(results)

Retrieved 5/5 chunks

Results for  哈利用什么魔法打败了索伦？

Rank 1:
    Distance: 0.432
    Similarity: 0.698
    Text: ## 第五章：魔法的融合

临战前夜，霍格沃茨的钟楼熄灯，取而代之的是城堡上空缓缓旋转的守护星光魔阵。夜色中笼罩着一股难以名状的寂静，仿佛整个世界都屏息等待。
...

Rank 2:
    Distance: 0.439
    Similarity: 0.695
    Text: # 魔戒与魔杖：两个世界的交汇

## 第一章：神秘的传送门

霍格沃茨的禁林，夜色正浓，月光从浓密树冠的缝隙中洒落，投下斑驳的银色光影。空气中弥漫着湿润的苔藓...

Rank 3:
    Distance: 0.471
    Similarity: 0.680
    Text: 火焰之眼骤然收缩、崩塌，在空中发出最后一声爆响后，化作万千光点飘散。半兽人军团仿佛失去灵魂的傀儡，纷纷倒地，化为虚无。

天边第一道金光破晓。

哈利缓缓放下魔...

Rank 4:
    Distance: 0.480
    Similarity: 0.675
    Text: “还有我，”赫敏抬起头，“如果允许，我想继续研究裂缝魔法，也许这会是巫师历史新的开端。”

邓布利多微微一笑：“霍格沃茨的图书馆，将永远为你敞开。”

甘道夫从...

Rank 5:
    Distance: 0.483
    Similarity: 0.674
    Text: “准备好了？”赫敏看着他。

“其实……没有。”罗恩苦笑一声，“但我们也没得选了。”

赫敏点头，挥杖落下最后一道光线。

整个魔法阵骤然亮起，泛出如水般的银光...


# Part 4: Rerank

In [6]:
from sentence_transformers import CrossEncoder

def rerank(
    query: str, 
    retrieved_results: List[dict], 
    top_k: int
) -> List[dict]:
    cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
    
    # Extract documents from retrieve() results
    chunks = [result['document'] for result in retrieved_results]
    pairs = [(query, chunk) for chunk in chunks]
    scores = cross_encoder.predict(pairs)
    
    # Preserve full result dict with rerank score
    for result, score in zip(retrieved_results, scores):
        result['rerank_score'] = float(score)
    
    # Sort by rerank score
    retrieved_results.sort(key=lambda x: x['rerank_score'], reverse=True)
    
    return retrieved_results[:top_k]

## Rerank Testing

In [7]:
retrieved = retrieve(query, top_k=5)
reranked = rerank(query, retrieved, top_k=3)

for i, result in enumerate(reranked):
    print(f"[{i}] Rerank: {result['rerank_score']:.3f} | {result['document'][:80]}\n")

Retrieved 5/5 chunks


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: cross-encoder/mmarco-mMiniLMv2-L12-H384-v1
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[0] Rerank: 3.363 | “准备好了？”赫敏看着他。

“其实……没有。”罗恩苦笑一声，“但我们也没得选了。”

赫敏点头，挥杖落下最后一道光线。

整个魔法阵骤然亮起，泛出如水般的银光

[1] Rerank: -0.106 | # 魔戒与魔杖：两个世界的交汇

## 第一章：神秘的传送门

霍格沃茨的禁林，夜色正浓，月光从浓密树冠的缝隙中洒落，投下斑驳的银色光影。空气中弥漫着湿润的苔藓

[2] Rerank: -0.479 | ## 第五章：魔法的融合

临战前夜，霍格沃茨的钟楼熄灯，取而代之的是城堡上空缓缓旋转的守护星光魔阵。夜色中笼罩着一股难以名状的寂静，仿佛整个世界都屏息等待。




# Part 5: LLM Generation

In [8]:
from dotenv import load_dotenv
from google import genai
from typing import List

load_dotenv()
google_client = genai.Client()

def generate(query: str, chunks: List[str]) -> str:
    # Combine retrieved chunks into context
    context = "\n\n".join([f"[片段 {i+1}]\n{chunk}" for i, chunk in enumerate(chunks)])
    
    prompt = f"""你是一个专业的小说问答助手。请基于以下提供的小说片段来回答用户的问题。

小说片段:
{context}

用户问题: {query}

请根据上述片段提供准确、详细的回答。不要编造信息。如果片段中没有足够信息回答问题,请说明。"""

    response = google_client.models.generate_content(
        model='gemini-2.5-flash',
        contents=prompt
    )
    
    return response.text

## LLM Generation Testing

In [9]:
# Usage
query = "哈利用什么魔法打败了索伦？"
retrieved = retrieve(query, top_k=5)
reranked = rerank(query, retrieved, top_k=3)
reranked_chunks = [result['document'] for result in reranked]

answer = generate(query, reranked_chunks)
print(answer)

Retrieved 5/5 chunks


Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

XLMRobertaForSequenceClassification LOAD REPORT from: cross-encoder/mmarco-mMiniLMv2-L12-H384-v1
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


根据小说片段提供的信息，哈利利用他的**光明守护神**（巨大的银色凤凰形态）来打败索伦。

这个光明守护神并非普通的守护神咒，它是在甘道夫的帮助下得到强化的：

1.  甘道夫将“阿尔诺之焰”（中土最纯净的圣光）汇入哈利的手心，并让其融入哈利的魔杖。
2.  哈利通过集中精神，回想起爱和希望（父母、西里斯、邓布利多、朋友），最终成功召唤出“一道巨大的银色凤凰”。
3.  这只凤凰的羽毛上镶嵌着星光般的符文，胸口蕴藏着甘道夫圣光的核心，它既是哈利的守护神，也是光明本身，代表了“两个世界希望的共鸣”。

在最终的战斗中，这股力量与**两位巫师（甘道夫和邓布利多）的联合光柱**以及**甘道夫法杖中的太阳之火**汇聚成一道三色神光，共同灼穿了索伦燃烧的瞳孔，使得黑暗开始崩塌。
